In [14]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 7.7 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=8204216bc9990b0741eeb467632a5a9c5d07db8ac34bf5efa7fbe6f587720270
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [15]:
from statsmodels.tsa.seasonal import seasonal_decompose
import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, kpss
import matplotlib.pyplot as plt
import yfinance as yf
from pandas_datareader import data, wb
import pandas_datareader as pdr
from statsmodels.tsa.stattools import kpss

[종목코드 받아오기]
---------

In [16]:
kospicode = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&marketType=stockMkt', header=0)[0]
kosdaqcode = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&marketType=kosdaqMkt', header=0)[0]

kospicode['시장구분'] = 'KS'
kosdaqcode['시장구분'] = 'KQ'

print('\n', kospicode.head())
print('\n', kosdaqcode.head())


          회사명    종목코드           업종  ...                    홈페이지     지역 시장구분
0      DRB동일    4840     고무제품 제조업  ...     http://drbworld.com  부산광역시   KS
1        DSR  155660  1차 비철금속 제조업  ...      http://www.dsr.com  부산광역시   KS
2         GS   78930       기타 금융업  ...                     NaN  서울특별시   KS
3      GS글로벌    1250    상품 종합 도매업  ...  http://www.gsgcorp.com  서울특별시   KS
4  HDC현대산업개발  294870       건물 건설업  ...  http://www.hdc-dvp.com  서울특별시   KS

[5 rows x 10 columns]

      회사명    종목코드               업종  ...                     홈페이지     지역 시장구분
0     3S   60310         전자부품 제조업  ...     http://www.3sref.com  서울특별시   KQ
1   EDGC  245620  자연과학 및 공학 연구개발업  ...      http://www.edgc.com  인천광역시   KQ
2   ES큐브   50120         생활용품 도매업  ...  http://www.escube.co.kr  서울특별시   KQ
3  HK이노엔  195940          의약품 제조업  ...    http://www.inno-n.com  서울특별시   KQ
4   JW신약   67290          의약품 제조업  ...    http://www.cwsy.co.kr  서울특별시   KQ

[5 rows x 10 columns]


In [17]:
krxcode = pd.concat([kospicode, kosdaqcode])
print('\n', krxcode.head(), '\n', krxcode.size, '\n')

krxcode = krxcode[['회사명','종목코드','시장구분']]
print(krxcode.head(), '\n', krxcode.tail())


          회사명    종목코드           업종  ...                    홈페이지     지역 시장구분
0      DRB동일    4840     고무제품 제조업  ...     http://drbworld.com  부산광역시   KS
1        DSR  155660  1차 비철금속 제조업  ...      http://www.dsr.com  부산광역시   KS
2         GS   78930       기타 금융업  ...                     NaN  서울특별시   KS
3      GS글로벌    1250    상품 종합 도매업  ...  http://www.gsgcorp.com  서울특별시   KS
4  HDC현대산업개발  294870       건물 건설업  ...  http://www.hdc-dvp.com  서울특별시   KS

[5 rows x 10 columns] 
 23270 

         회사명    종목코드 시장구분
0      DRB동일    4840   KS
1        DSR  155660   KS
2         GS   78930   KS
3      GS글로벌    1250   KS
4  HDC현대산업개발  294870   KS 
           회사명    종목코드 시장구분
1504    한라IMS   92460   KQ
1505  한스바이오메드   42520   KQ
1506     현대사료   16790   KQ
1507   휴맥스홀딩스   28080   KQ
1508       힘스  238490   KQ


In [18]:
def get_code(stockname):
  targetcode = krxcode.loc[krxcode['회사명']==stockname,['종목코드']]
  targetcode = '%06d' %int(targetcode.values)

  return targetcode

In [20]:
targetCode = get_code('삼성전자')
print(targetCode)

005930


[yahoo finance data]
------

In [ ]:
df_hm = yf.download(str(targetCode)+'.KS', start='2010-01-01', end='2021-08-31')

[*********************100%***********************]  1 of 1 completed


In [ ]:
df_hm.reset_index(drop=False, inplace=True)
df_hm.rename(columns={'Date':'ds','Adj Close':'y'},inplace=True)
df_hm = df_hm.drop(['Open','High','Low','Close','Volume'], axis =1)
df_hm

,ds,y
0,2010-01-04,13796.411133
1,2010-01-05,13973.287109
2,2010-01-06,14282.822266
3,2010-01-07,13973.287109
4,2010-01-08,14150.164062
...,...,...
2857,2021-08-25,282000.000000
2858,2021-08-26,284500.000000
2859,2021-08-27,294500.000000
2860,2021-08-30,291000.000000


In [ ]:
def adf_test(x):
    
    indices = ['Test Statistic', 'p-value',
               '# of Lags Used', '# of Observations Used']
    adf_test = adfuller(x, autolag='AIC')
    # if 'AIC' (default) or 'BIC', then the number of lags is chosen to minimize the corresponding information criterion
    # AIC가 디폴트 값인데, lag의 수준을 최적화하는 것으로 보면 됨.
    # H0은 non-stationary
    # H1은 stationary
    # p-value < alpha  >> H0 기각 / H1 채택
    results = pd.Series(adf_test[0:4], index=indices)
    for key, value in adf_test[4].items():
        results[f'Critical Value ({key})'] = value

    return results

In [ ]:
def kpss_test(x, h0_type='c'):

    indices = ['Test Statistic', 'p-value', '# of Lags']
    kpss_test = kpss(x, regression=h0_type)

    # regrssion = c or ct로 설정할 수 있음. 
    # c=contant : 해당 데이터가 정상성이 있다. 귀무가설에 해당함. (H0 = stationary)
    # ct=data is stationary around a trend lags : 해당 데이터에 트랜드를 제거하는 것이 정상성을 만드는 방법
    results = pd.Series(kpss_test[0:3], index=indices)
    for key, value in kpss_test[3].items():
        results[f'Critical Value ({key})'] = value

    return results

In [ ]:
# def kpss_test(timeseries):
#     print ('Results of KPSS Test:')
#     kpsstest = kpss(timeseries, regression='c', nlags="auto")
#     kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','Lags Used'])
#     for key,value in kpsstest[3].items():
#         kpss_output['Critical Value (%s)'%key] = value
#     print (kpss_output)

In [ ]:
adf_test(df_hm.y)

Test Statistic              -2.287332
p-value                      0.176086
# of Lags Used               3.000000
# of Observations Used    2870.000000
Critical Value (1%)         -3.432631
Critical Value (5%)         -2.862548
Critical Value (10%)        -2.567306
dtype: float64

In [ ]:
kpss_test(df_hm.y)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/stattools.py:1685: FutureWarning: The behavior of using lags=None will change in the next release. Currently lags=None is the same as lags='legacy', and so a sample-size lag length is used. After the next release, the default will change to be the same as lags='auto' which uses an automatic lag length selection method. To silence this warning, either use 'auto' or 'legacy'
  warn(msg, FutureWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/stattools.py:1709: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)


Test Statistic            1.030335
p-value                   0.010000
# of Lags                28.000000
Critical Value (10%)      0.347000
Critical Value (5%)       0.463000
Critical Value (2.5%)     0.574000
Critical Value (1%)       0.739000
dtype: float64